In [15]:
import boto3
import sagemaker
from sagemaker import Session
from sagemaker.session import TrainingInput
from sagemaker.tensorflow import TensorFlow as tf
from sagemaker.tensorflow import TensorFlowModel
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, ContinuousParameter, CategoricalParameter
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import tensorflow
import s3fs
import numpy as np
import json
RANDOM_SEED = 0
sagemaker.__version__

'2.245.0'

In [19]:
fs = s3fs.S3FileSystem()
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
s3 = boto3.client("s3")
bucket = sess.default_bucket()   # will be sagemaker-ap-southeast-2-838084669510
prefix = "aiornot"
print(f"S3 Bucket: f{bucket}")

s3_small_train_path = f"s3://{bucket}/{prefix}/small_train/small_train.npz"
s3_train_path = f"s3://{bucket}/{prefix}/train/train.npz"
s3_test_path = f"s3://{bucket}/{prefix}/test/test.npz"

s3_output_location = f"s3://{bucket}/{prefix}/model_output"

small_train_input = TrainingInput(s3_small_train_path, content_type="application/x-npz")
train_input = TrainingInput(s3_train_path, content_type="application/x-npz")
test_input = TrainingInput(s3_test_path, content_type="application/x-npz")


S3 Bucket: fsagemaker-ap-southeast-2-838084669510


In [21]:
estimator = tf(
    entry_point="train.py",
    source_dir="src",   # where train.py and model_def.py live
    role=role,
    # use_spot_instances=True,  # save money
    instance_type="ml.c5.2xlarge",
    instance_count=1,
    framework_version="2.14",
    py_version="py310",
    hyperparameters={
        "epochs": 1,
        "height": 512,
        "width": 512,
        "channels": 3
    },
    output_path=s3_output_location
)

# Step 2: define search space
hyperparameter_ranges = {
    # numeric / continuous
    "learning-rate": ContinuousParameter(1e-4, 1e-2, scaling_type="Logarithmic"),
    "dropout-rate": ContinuousParameter(0.0, 0.5),                 # if use-dropout=true
    # integer choices for layer sizes
    "batch-size": IntegerParameter(4, 8),                         # change range to suit memory
    "conv1-filters": IntegerParameter(16, 128),
    "conv2-filters": IntegerParameter(32, 256),
    "dense-units": IntegerParameter(64, 512),
    # categorical choices
    "pooling": CategoricalParameter(["max", "avg"]),
    "use-dropout": CategoricalParameter(["true", "false"]),
    "optimizer": CategoricalParameter(["adam", "adagrad"]),
}

# Step 3: define regex to extract val_accuracy from train.py logs
metric_definitions = [
    {"Name": "val_auc",       "Regex": "val_auc: ([0-9\\.]+)"},
    {"Name": "val_f1",        "Regex": "val_f1: ([0-9\\.]+)"},
    {"Name": "val_precision", "Regex": "val_precision: ([0-9\\.]+)"},
    {"Name": "val_recall",    "Regex": "val_recall: ([0-9\\.]+)"},
    {"Name": "val_accuracy",  "Regex": "val_accuracy: ([0-9\\.]+)"},
]

# Step 4: set up the tuner
tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name="val_f1",
    # strategy='Hyperband',
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metric_definitions,
    max_parallel_jobs=2,
    objective_type="Maximize",
    early_stopping_type="Auto",
    max_jobs=1,
    base_tuning_job_name="ph-15",
)

# Step 5: launch it using small_train.npz for both train + val
tuner.fit({
    "train": small_train_input,
    "test": test_input,
})


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


........................................................................!


In [18]:
tuning_job_name = "ph-14-250814-2347-001-17ca7129"
print("Best training job:", tuning_job_name)
best_estimator = tuner.best_estimator()
sm = boto3.client("sagemaker")
tj = sm.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)
best = tj.get("BestTrainingJob", {})
best_training_job_name = best.get("TrainingJobName")
print("Best training job (from tuning):", best_training_job_name)


Best training job: ph-14-250814-2347-001-17ca7129

2025-08-14 23:57:55 Starting - Preparing the instances for training
2025-08-14 23:57:55 Downloading - Downloading the training image
2025-08-14 23:57:55 Training - Training image download completed. Training in progress.
2025-08-14 23:57:55 Uploading - Uploading generated training model
2025-08-14 23:57:55 Completed - Resource released due to keep alive period expiry


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 print("Best training job:", tuning_job_name)                                                 │
│   3 best_estimator = tuner.best_estimator()                                                      │
│   4 sm = boto3.client("sagemaker")                                                               │
│ ❱ 5 tj = sm.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)     │
│   6 best = tj.get("BestTrainingJob", {})                                                         │
│   7 best_training_job_name = best.get("TrainingJobName")                                         │
│   8 print("Best training job (from tuning):", best_training_job_name)                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ResourceNotFound: An error occurred (ResourceNotFound) when calling the DescribeHyperParameterTuningJob operation: 
Amazon SageMaker can't find a tuning job called ph-14-250814-2347-001-17ca7129. Type another name.

In [13]:
model_name = "tensorflow-training-2025-08-14-04-53-54-648"
model_s3_path = f"s3://{bucket}/{model_name}/source/sourcedir.tar.gz"
output_s3 = f"s3://{bucket}/batch-output/tf-transform-{model_name}/"
input_s3_uri = f"s3://{bucket}/aiornot/test/test.npz"
endpoint_name = 'model-endpoint'

model = TensorFlowModel(
    model_data=model_s3_path,
    role=role,
    entry_point="inference.py",
    source_dir="src",
    framework_version="2.14",
    sagemaker_session=sess
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.xlarge",
    endpoint_name=endpoint_name,
    wait=True
)

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()
predictor.content_type = "application/json"
predictor.accept = "application/json"

local_tmp = "/tmp/tiny_test.npz"
s3_key = "aiornot/tiny_test/tiny_test.npz"
s3.download_file(bucket, s3_key, local_tmp)

npz = np.load(local_tmp, allow_pickle=False)
print("npz keys:", npz.files)   # should print ['image','label']
images = npz["image"]           # shape (20, 512,512,3)
labels = npz["label"]           # shape (20,)

------------------------------------------*

Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-endpoint


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:16                                                                                   │
│                                                                                                  │
│   13 │   sagemaker_session=sess                                                                  │
│   14 )                                                                                           │
│   15                                                                                             │
│ ❱ 16 predictor = model.deploy(                                                                   │
│   17 │   initial_instance_count=1,                                                               │
│   18 │   instance_type="ml.c5.xlarge",                                                           │
│   19 │   endpoint_name=endpoint_name,                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/tensorflow/model.py:369 in deploy              │
│                                                                                                  │
│   366 │   │   │   msg = "The TensorFlow version %s doesn't support EIA." % self.framework_vers   │
│   367 │   │   │   raise AttributeError(msg)                                                      │
│   368 │   │                                                                                      │
│ ❱ 369 │   │   return super(TensorFlowModel, self).deploy(                                        │
│   370 │   │   │   initial_instance_count=initial_instance_count,                                 │
│   371 │   │   │   instance_type=instance_type,                                                   │
│   372 │   │   │   serializer=serializer,                                                         │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/model.py:1814 in deploy                        │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/session.py:6033 in                             │
│ endpoint_from_production_variants                                                                │
│                                                                                                  │
│   6030 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6031 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6032 │   │                                                                                     │
│ ❱ 6033 │   │   return self.create_endpoint(                                                      │
│   6034 │   │   │   endpoint_name=name,                                                           │
│   6035 │   │   │   config_name=name,                       

In [6]:
sm = boto3.client("sagemaker")
training_job_name = "ph-12-250814-0453-001-32522441"
training_job = sm.describe_training_job(TrainingJobName=training_job_name)
print("S3ModelArtifacts:", training_job["ModelArtifacts"]["S3ModelArtifacts"])
print("TrainingJobStatus:", training_job.get("TrainingJobStatus"))
print("FailureReason:", training_job.get("FailureReason"))
print("ModelArtifacts S3 URI:", training_job.get("ModelArtifacts", {}).get("S3ModelArtifacts"))

S3ModelArtifacts: s3://sagemaker-ap-southeast-2-838084669510/ph-12-250814-0453-001-32522441/output/model.tar.gz
TrainingJobStatus: Completed
FailureReason: None
ModelArtifacts S3 URI: s3://sagemaker-ap-southeast-2-838084669510/ph-12-250814-0453-001-32522441/output/model.tar.gz


In [7]:
s3_uri = "s3://sagemaker-ap-southeast-2-838084669510/ph-12-250814-0453-001-32522441/output/model.tar.gz"
bucket, key = s3_uri.replace("s3://","").split("/", 1)
prefix = key.rsplit("/",1)[0] + "/"

paginator = s3.get_paginator("list_objects_v2")
found = False
print("Listing objects under:", prefix)
for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
    for obj in page.get("Contents", []):
        print(obj["Key"], obj["Size"])
        if obj["Key"] == key:
            found = True

print("\nmodel.tar.gz present?:", found)

Listing objects under: ph-12-250814-0453-001-32522441/output/

model.tar.gz present?: False


In [9]:
s3 = boto3.client("s3")
bucket = "sagemaker-ap-southeast-2-838084669510"

paginator = s3.get_paginator("list_objects_v2")
matches = []
for page in paginator.paginate(Bucket=bucket):
    for obj in page.get("Contents", []):
        k = obj["Key"]
        if k.endswith("saved_model.pb"):
            matches.append(k)
            print("Found saved_model.pb at:", k)

print("Total saved_model.pb files found:", len(matches))

Found saved_model.pb at: tensorflow-training-2025-08-14-01-17-19-905/model/ph-08-250814-0117-001-fa1ff411/model/1/saved_model.pb
Found saved_model.pb at: tensorflow-training-2025-08-14-04-13-29-059/model/ph-10-250814-0413-001-7b46a067/model/1/saved_model.pb
Found saved_model.pb at: tensorflow-training-2025-08-14-04-53-54-648/model/ph-12-250814-0453-001-32522441/model/1/saved_model.pb
Total saved_model.pb files found: 3
